In [1]:
import pandas as pd


from google.colab import drive
drive.mount('/content/drive')
splits = pd.read_csv('../data/data_splits.csv',
                     dtype={
    'book': 'string',
    'chapter': 'string',
    'split': 'string'
})

Mounted at /content/drive


In [2]:
from tqdm import tqdm
import re

In [3]:
devs = splits[splits['split'] == 'DEV']
trains = splits[splits['split'] == 'TRAIN']
tests = splits[splits['split'] == 'TEST']

### Objective: get aligned dev, test, and train fragments.
x_l3.frags.txt, and x_l3.frags.pnx.txt. Do things per book number and following the splits. Structure: {dev/train/test}/{book_no}/{chapter}_level.frags.pnx.txt

In [4]:
base_path = '../data/splits'
dev_pattern = '/dev/{}/{}.{}.txt'
train_pattern = '/train/{}/{}.{}.txt'
test_pattern = '/test/{}/{}.{}.txt'

frags = 'frags'
frags_pnx = 'frags.pnx'

columns = ['l3', 'l4', 'l5']

def get_all(pattern, splits, frags = frags_pnx):
  all = pd.DataFrame(columns = columns)
  for _, row in tqdm(splits.iterrows()):
    with open(base_path + pattern.format(row['book'], row['chapter'], frags)) as f:
      all_l5 = f.read().split('\n')
    with open(base_path + pattern.format(row['book'], row['chapter'] + "_l3", frags)) as f:
      all_l3 = f.read().split('\n')
    with open(base_path + pattern.format(row['book'], row['chapter'] + "_l4", frags)) as f:
      all_l4 = f.read().split('\n')

    temp = pd.DataFrame({'l3': all_l3, 'l4': all_l4, 'l5': all_l5})

    all = pd.concat([all, temp])

  return all




In [5]:
all_dev = get_all(dev_pattern, devs)
all_train = get_all(train_pattern, trains)
all_test = get_all(test_pattern, tests)


20it [00:27,  1.36s/it]
104it [02:06,  1.21s/it]
22it [00:24,  1.10s/it]


In [6]:
all_test

,l3,l4,l5
0,ولكنها تحركت !,ولكنها تحركت !,ولكنها تحركت !
1,إما لأنها أحست به وإما لأن الوقفة أتعبتها أو أ...,إما لأنها أحست به وإما لأن الوقفة أتعبتها أو أ...,إما لأنها أحست به وإما لأن الوقفة أتعبتها أو أ...
2,فرأته فصبغ الدم وجهها وارتدت ،,فرأته فصبغ الدم وجهها وارتدت ،,فرأته فصبغ الدم وجهها وارتدت ،
3,ولكنها لم تعبس له وقالت وفي عينها نظرة عتب ورض...,ولكنها لم تعبس له وقالت وفي عينها نظرة عتب ورض...,ولكنها لم تتجهم له وقالت وفي عينها نظرة عتب ور...
4,ألك هنا كثير ؟,ألك هنا كثير ؟,ألك هنا كثير ؟
...,...,...,...
204,ولكنك لم توضح لي صفات هؤلاء الشمامين .,ولكنك لم توضح لي صفات هؤلاء الشمامين .,ولكنك لم توضح لي صفات هؤلاء الشمامين .
205,س,س,س
206,:,:,:
207,إلى سهرة الغد تصبحون على خير .,إلى سهرة الغد تصبحون على خير .,إلى سهرة الغد تصبحون على خير .


In [7]:
print(len(all_dev), len(all_test), len(all_train))

2989 3296 14436


In [8]:
devs

,book,chapter,split
0,0,1,DEV
9,1,1,DEV
10,1,2,DEV
20,2,1,DEV
29,3,1.1,DEV
40,4,1.1,DEV
51,5,1.1,DEV
58,6,1.1,DEV
65,7,1.1,DEV
75,8,1,DEV


Collapse and individualise the splits

In [25]:
def collapse_and_individualise(orig):
  df = pd.DataFrame(columns =['text','l4','l3', 'level'])
  for _, row in tqdm(orig.iterrows()):
    if row['l3'] == row['l4'] and row['l4'] == row['l5']: # 333
      df = df._append({'text': row['l3'], 'l4': '\'=orig', 'l3': '\'=orig', 'level': 3}, ignore_index=True)

    elif row['l3'] == row['l4'] and row['l4'] != row['l5']: # 533
      df = df._append({'text': row['l5'], 'l4': '\'=l3', 'l3': row['l3'], 'level': 5}, ignore_index=True)
      #df = df._append({'text': row['l3'], 'level': 3}, ignore_index=True)

    elif row['l3'] != row['l4'] and row['l4'] == row['l5']: # 443
      df = df._append({'text': row['l4'], 'l4': '\'=orig', 'l3': row['l3'], 'level': 4}, ignore_index=True)
      #df = df._append({'text': row['l3'], 'level': 3}, ignore_index=True)

    elif row['l3'] != row['l4'] and row['l4'] != row['l5']: # 543
      df = df._append({'text': row['l5'], 'l4': row['l4'], 'l3': row['l3'], 'level': 5}, ignore_index=True)
      #df = df._append({'text': row['l4'], 'level': 4}, ignore_index=True)
      #df = df._append({'text': row['l3'], 'level': 3}, ignore_index=True)

  return df

In [26]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
dev_levelled = collapse_and_individualise(all_dev)
train_levelled = collapse_and_individualise(all_train)
test_levelled = collapse_and_individualise(all_test)

2989it [00:04, 736.31it/s]
14436it [00:25, 572.05it/s]
3296it [00:04, 732.02it/s]


Take punctuation-only fragments out

In [28]:
def prune_punctuation(fragment):
  all = fragment.split(' ')
  all = [a for a in all if not re.fullmatch(r'^[–•«»!,.:\u060C\u061B\u061F\u2026]+$', a)]
  if len(all) == 0:
    return 'DELETEME'
  else:
    return ' '.join(all)

def cleanse(df):
  df['text'] = df.apply(lambda row: prune_punctuation(row['text']), axis=1)
  df = df[df['text'] != 'DELETEME']
  return df

In [29]:
dev_levelled = cleanse(dev_levelled)
train_levelled = cleanse(train_levelled)
test_levelled = cleanse(test_levelled)

In [30]:
dev_levelled

,text,l4,l3,level
0,إبراهيم الكاتب,'=orig,'=orig,3
1,إبراهيم عبد القادر المازني,'=orig,'=orig,3
2,الفصل الأول,'=orig,'=orig,3
3,وكان مساء,'=orig,'=orig,3
4,شوشو فتاة يقول لك جسمها إنها ناهزت التاسعة عشرة,'=l3,شوشو فتاة يقول لك جسمها إنها اقتربت من التاسعة...,5
...,...,...,...,...
2984,فوقفوا جميعا,'=orig,'=orig,3
2985,وقالوا بصوت واحد حفظ الله الملك العظيم وأدام ت...,'=orig,'=orig,3
2986,ثم انصرفوا,'=orig,'=orig,3
2987,وفي كل نفس هواجسها,'=orig,وفي كل نفس مخاوفها .,4


In [31]:
dev_levelled = dev_levelled[dev_levelled.apply(lambda x: x['text'] != '', axis = 1)]


In [32]:
dev_levelled

,text,l4,l3,level
0,إبراهيم الكاتب,'=orig,'=orig,3
1,إبراهيم عبد القادر المازني,'=orig,'=orig,3
2,الفصل الأول,'=orig,'=orig,3
3,وكان مساء,'=orig,'=orig,3
4,شوشو فتاة يقول لك جسمها إنها ناهزت التاسعة عشرة,'=l3,شوشو فتاة يقول لك جسمها إنها اقتربت من التاسعة...,5
...,...,...,...,...
2983,فإلى الغد في مثل هذا الوقت,'=orig,'=orig,3
2984,فوقفوا جميعا,'=orig,'=orig,3
2985,وقالوا بصوت واحد حفظ الله الملك العظيم وأدام ت...,'=orig,'=orig,3
2986,ثم انصرفوا,'=orig,'=orig,3


Save the splits

In [ ]:
base_path = '../data/'

dev_levelled.to_csv(base_path + 'dev_levelled.csv')

train_levelled.to_csv(base_path + 'train_levelled.csv')

test_levelled.to_csv(base_path + 'test_levelled.csv')

dev_levelled.to_csv(base_path + 'dev_levelled_w_parallels.csv')


### Same thing but NOT punct tokenized

In [ ]:
all_dev_np = get_all(dev_pattern, devs, frags)
all_train_np = get_all(train_pattern, trains, frags)
all_test_np = get_all(test_pattern, tests, frags)

dev_levelled_np = collapse_and_individualise(all_dev_np)
train_levelled_np = collapse_and_individualise(all_train_np)
test_levelled_np = collapse_and_individualise(all_test_np)

dev_levelled_np = cleanse(dev_levelled_np)
train_levelled_np = cleanse(train_levelled_np)
test_levelled_np = cleanse(test_levelled_np)


dev_levelled_np.to_csv(base_path + 'dev_levelled_np.csv')

train_levelled_np.to_csv(base_path + 'train_levelled_np.csv')

test_levelled_np.to_csv(base_path + 'test_levelled_np.csv')

In [ ]:
dev_levelled_np